In [1]:
import pymongo
import json
import pandas as pd
from datetime import datetime, timedelta
from pandas import DataFrame
import matplotlib.pyplot as plt
from datetime import time
from tqdm import tqdm

myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["studentlife"]
collection = mydb["audio"]

In [2]:
df_audio = pd.read_csv('/Users/ohidabinteamin/Documents/Stress Prediction Project Three Datasets/StudentLife/week 01/Audio/audio_data_studentlife.csv')

In [3]:
df_audio.head(3)

,Unnamed: 0,timestamp,audio inference,uid,date
0,0,2013-03-27 04:00:00,0,u59,2013-03-27
1,1,2013-03-27 04:00:01,0,u59,2013-03-27
2,2,2013-03-27 04:00:02,0,u59,2013-03-27


In [4]:
del df_audio['Unnamed: 0']

In [5]:
morning_start_time = time(6, 0, 0)     # 6:00 AM
afternoon_start_time = time(12, 0, 0)  # 12:00 PM
evening_start_time = time(18, 0, 0)    # 6:00 PM
night_start_time = time(0, 0, 0)       # 12:00 AM (midnight)

In [6]:
def categorize_time_interval(timestamp):
    current_time = timestamp.time()
    
    if morning_start_time <= current_time < afternoon_start_time:
        return "morning"
    elif afternoon_start_time <= current_time < evening_start_time:
        return "afternoon"
    elif evening_start_time <= current_time <= time(23, 59, 59):
        return "evening"
    else:
        return "night"

In [7]:
df_audio

,timestamp,audio inference,uid,date
0,2013-03-27 04:00:00,0,u59,2013-03-27
1,2013-03-27 04:00:01,0,u59,2013-03-27
2,2013-03-27 04:00:02,0,u59,2013-03-27
3,2013-03-27 04:00:03,0,u59,2013-03-27
4,2013-03-27 04:00:04,0,u59,2013-03-27
...,...,...,...,...
93690396,2013-05-30 19:59:47,0,u54,2013-05-30
93690397,2013-05-30 19:59:48,0,u54,2013-05-30
93690398,2013-05-30 19:59:49,0,u54,2013-05-30
93690399,2013-05-30 19:59:50,0,u54,2013-05-30


In [8]:
df_audio['timestamp'] = pd.to_datetime(df_audio['timestamp'])
df_audio['time_interval'] = df_audio['timestamp'].apply(categorize_time_interval)
df_audio = df_audio.sort_values(by=['uid', 'timestamp'])

In [9]:
df_audio.head(3)

,timestamp,audio inference,uid,date,time_interval
21603692,2013-03-27 04:02:21,0,u00,2013-03-27,night
21603693,2013-03-27 04:02:22,0,u00,2013-03-27,night
21603694,2013-03-27 04:02:23,0,u00,2013-03-27,night


In [10]:
df_audio['audio inference'].value_counts()

audio inference
0    50344148
1    25595732
2    17750521
Name: count, dtype: int64

In [11]:
df_data = DataFrame()

In [12]:
audio_uid = ['u00', 'u01', 'u02', 'u03', 'u04', 'u05', 'u07', 'u08', 'u09', 'u10', 'u12', 'u13', 'u14', 'u15', 'u16',
                'u17', 'u18', 'u19', 'u20', 'u22', 'u23', 'u24', 'u25', 'u27', 'u30', 'u31', 'u32', 'u33', 'u34', 'u35',
                'u36', 'u39', 'u41', 'u42', 'u43', 'u44', 'u45', 'u46', 'u47', 'u49', 'u50', 'u51', 'u52', 'u53', 'u54',
                'u56', 'u57', 'u58', 'u59']

In [13]:
def classify_audio(audio_inference):
    if audio_inference == 0:
        return "silent"
    elif audio_inference == 1:
        return "voice"
    elif audio_inference == 2:
        return "noise"
    else:
        return "unknown"

In [14]:
df_audio['audio'] = df_audio['audio inference'].apply(classify_audio)
df_audio = df_audio.drop("audio inference", axis=1)
df_audio.head(3)

,timestamp,uid,date,time_interval,audio
21603692,2013-03-27 04:02:21,u00,2013-03-27,night,silent
21603693,2013-03-27 04:02:22,u00,2013-03-27,night,silent
21603694,2013-03-27 04:02:23,u00,2013-03-27,night,silent


In [15]:
df_audio['audio'].value_counts()

audio
silent    50344148
voice     25595732
noise     17750521
Name: count, dtype: int64

In [16]:
df_audio.head(5)

,timestamp,uid,date,time_interval,audio
21603692,2013-03-27 04:02:21,u00,2013-03-27,night,silent
21603693,2013-03-27 04:02:22,u00,2013-03-27,night,silent
21603694,2013-03-27 04:02:23,u00,2013-03-27,night,silent
21603695,2013-03-27 04:02:24,u00,2013-03-27,night,silent
21603696,2013-03-27 04:02:25,u00,2013-03-27,night,silent


In [17]:
def process_audio_data_for_uid(uid, df_audio):
    columns = ['uid', 'date',
               'morning_noise_duration', 'morning_silent_duration', 'morning_voice_duration',
               'afternoon_noise_duration', 'afternoon_silent_duration', 'afternoon_voice_duration',
               'evening_noise_duration', 'evening_silent_duration', 'evening_voice_duration',
               'night_noise_duration', 'night_silent_duration', 'night_voice_duration']
    result_df = pd.DataFrame(columns=columns)
    user_df = df_audio[df_audio['uid'] == uid]

    for date in user_df['date'].unique():
        date_df = user_df[user_df['date'] == date]

        audios = {
            'noise_morning': 0, 'silent_morning': 0, 'voice_morning': 0,
            'noise_afternoon': 0, 'silent_afternoon': 0, 'voice_afternoon': 0,
            'noise_evening': 0, 'silent_evening': 0, 'voice_evening': 0,
            'noise_night': 0, 'silent_night': 0, 'voice_night': 0,
        }

        grouped_by_interval = date_df.groupby('time_interval')

        for interval, interval_df in grouped_by_interval:
            start_time = None
            previous_audio = None

            first_record = pd.to_datetime(interval_df.iloc[0]['timestamp'])
            init_missing_duration = None

            if interval == "morning":
                manual_start_morning = pd.to_datetime(f"{date} 06:00:00")
                init_missing_duration = (first_record - manual_start_morning).total_seconds()
            elif interval == "afternoon":
                manual_start_afternoon = pd.to_datetime(f"{date} 12:00:00")
                init_missing_duration = (first_record - manual_start_afternoon).total_seconds()
            elif interval == "evening":
                manual_start_evening = pd.to_datetime(f"{date} 18:00:00")
                init_missing_duration = (first_record - manual_start_evening).total_seconds()
            elif interval == "night":
                manual_start_night = pd.to_datetime(f"{date} 00:00:00")
                init_missing_duration = (first_record - manual_start_night).total_seconds()

            duration_hours = init_missing_duration / 3600

            for index, row in interval_df.iterrows():
                current_audio = row['audio']
                current_time = pd.to_datetime(row['timestamp'])

                if current_audio in ['noise', 'silent', 'voice']:
                    if current_audio == previous_audio:
                        if start_time is not None:
                            duration_seconds = (current_time - start_time).seconds
                            duration_hours = duration_seconds / 3600.0

                            update_variable = f"{previous_audio}_{interval}"
                            audios[update_variable] += duration_hours
                        start_time = current_time
                    else:
                        if start_time is not None:
                            duration_seconds = (current_time - start_time).seconds
                            duration_hours = duration_seconds / 3600.0
                        start_time = current_time

                    previous_audio = current_audio

            last_record = pd.to_datetime(interval_df.iloc[-1]['timestamp'])
            init_missing_duration = None

            if interval == "morning":
                manual_end_morning = pd.to_datetime(f"{date} 12:00:00")
                init_missing_duration = (manual_end_morning - last_record).total_seconds()
            elif interval == "afternoon":
                manual_end_afternoon = pd.to_datetime(f"{date} 18:00:00")
                init_missing_duration = (manual_end_afternoon - last_record).total_seconds()
            elif interval == "evening":
                manual_end_evening = pd.to_datetime(f"{date} 23:59:59")
                init_missing_duration = (manual_end_evening - last_record).total_seconds()
            elif interval == "night":
                manual_end_night = pd.to_datetime(f"{date} 06:00:00")
                init_missing_duration = (manual_end_night - last_record).total_seconds()

            duration_hours = init_missing_duration / 3600

        new_row = pd.DataFrame([{
            'uid': uid,
            'date': date,
            'morning_noise_duration': audios['noise_morning'],
            'morning_silent_duration': audios['silent_morning'],
            'morning_voice_duration': audios['voice_morning'],
            'afternoon_noise_duration': audios['noise_afternoon'],
            'afternoon_silent_duration': audios['silent_afternoon'],
            'afternoon_voice_duration': audios['voice_afternoon'],
            'evening_noise_duration': audios['noise_evening'],
            'evening_silent_duration': audios['silent_evening'],
            'evening_voice_duration': audios['voice_evening'],
            'night_noise_duration': audios['noise_night'],
            'night_silent_duration': audios['silent_night'],
            'night_voice_duration': audios['voice_night'],
        }])

        result_df = pd.concat([result_df, new_row], ignore_index=True)

    return result_df

In [18]:
for uid in tqdm(audio_uid):
    print(uid)
    df = process_audio_data_for_uid(str(uid), df_audio)
    frames = [df_data, df]
    df_data = pd.concat(frames, ignore_index=True)

  0%|                                                    | 0/49 [00:00<?, ?it/s]

u00


/var/folders/k8/745_jm4x0n35zzr1p5chpdfh0000gn/T/ipykernel_81702/3033597953.py:100: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result_df = pd.concat([result_df, new_row], ignore_index=True)
  2%|▉                                           | 1/49 [00:49<39:53, 49.87s/it]

u01


/var/folders/k8/745_jm4x0n35zzr1p5chpdfh0000gn/T/ipykernel_81702/3033597953.py:100: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result_df = pd.concat([result_df, new_row], ignore_index=True)
  4%|█▊                                          | 2/49 [01:40<39:27, 50.38s/it]

...

/var/folders/k8/745_jm4x0n35zzr1p5chpdfh0000gn/T/ipykernel_81702/3033597953.py:100: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result_df = pd.concat([result_df, new_row], ignore_index=True)
 96%|█████████████████████████████████████████▏ | 47/49 [31:15<01:24, 42.31s/it]

u58


/var/folders/k8/745_jm4x0n35zzr1p5chpdfh0000gn/T/ipykernel_81702/3033597953.py:100: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result_df = pd.concat([result_df, new_row], ignore_index=True)
 98%|██████████████████████████████████████████ | 48/49 [32:04<00:44, 44.24s/it]

u59


/var/folders/k8/745_jm4x0n35zzr1p5chpdfh0000gn/T/ipykernel_81702/3033597953.py:100: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result_df = pd.concat([result_df, new_row], ignore_index=True)
100%|███████████████████████████████████████████| 49/49 [33:38<00:00, 41.19s/it]


In [19]:
df_data.columns

Index(['uid', 'date', 'morning_noise_duration', 'morning_silent_duration',
       'morning_voice_duration', 'afternoon_noise_duration',
       'afternoon_silent_duration', 'afternoon_voice_duration',
       'evening_noise_duration', 'evening_silent_duration',
       'evening_voice_duration', 'night_noise_duration',
       'night_silent_duration', 'night_voice_duration'],
      dtype='object')

In [20]:
df_data.isnull().sum()

uid                          0
date                         0
morning_noise_duration       0
morning_silent_duration      0
morning_voice_duration       0
afternoon_noise_duration     0
afternoon_silent_duration    0
afternoon_voice_duration     0
evening_noise_duration       0
evening_silent_duration      0
evening_voice_duration       0
night_noise_duration         0
night_silent_duration        0
night_voice_duration         0
dtype: int64

In [21]:
len(df_data)

2883

In [22]:
df_data.head(3)

,uid,date,morning_noise_duration,morning_silent_duration,morning_voice_duration,afternoon_noise_duration,afternoon_silent_duration,afternoon_voice_duration,evening_noise_duration,evening_silent_duration,evening_voice_duration,night_noise_duration,night_silent_duration,night_voice_duration
0,u00,2013-03-27,0.209444,5.2475,0.178333,0.778333,2.723333,0.941944,1.540278,0.882778,2.009444,0.058611,1.567222,0.018056
1,u00,2013-03-28,0.202778,4.129722,0.114167,0.581389,3.404444,0.843889,0.634444,2.196944,2.015833,0.275833,4.764167,0.299722
2,u00,2013-03-29,0.481389,4.647778,0.190556,1.462778,1.797778,1.082778,0.885,1.690556,1.776389,1.125,2.342778,0.849444


In [23]:
df_data.to_csv('raw_audio_features_studentlife.csv')

In [24]:
len(df_data['uid'].unique())

49